In [5]:
import numpy as np
import cv2

# Harris 코너 검출 함수
def corner_harris():
    # 이미지를 그레이스케일로 로드
    src = cv2.imread('../data/img/aaa.jpg', cv2.IMREAD_GRAYSCALE)

    # 이미지 로드에 실패한 경우
    if src is None:
        print('Image load failed!')
        return

    # Harris 코너 검출 수행
    harris = cv2.cornerHarris(src, 3, 3, 0.04)
    # 검출된 결과를 0-255 범위로 정규화
    harris_norm = cv2.normalize(harris, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # 결과를 표시하기 위해 그레이스케일 이미지를 BGR로 변환
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

    # Harris 코너 검출 결과에서 코너점 찾기
    for y in range(harris_norm.shape[0]):
        for x in range(harris_norm.shape[1]):
            # 코너 강도가 임계값(50)보다 큰 경우
            if harris_norm[y, x] > 50:
                # 해당 점이 주변 점들보다 강도가 큰지 확인하여 코너점인지 판단
                if (harris[y, x] > harris[y-1, x] and
                    harris[y, x] > harris[y-1, x] and
                    harris[y, x] > harris[y-1, x] and
                    harris[y, x] > harris[y-1, x]):
                    # 코너점을 빨간 원으로 표시
                    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

    # 원본 이미지, 정규화된 Harris 결과, 코너점이 표시된 이미지를 화면에 표시
    cv2.imshow('src', src)
    cv2.imshow('harris_norm', harris_norm)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

# FAST 코너 검출 함수
def corner_fast():
    # 이미지를 그레이스케일로 로드
    src = cv2.imread('../data/img/eee.png', cv2.IMREAD_GRAYSCALE)

    # 이미지 로드에 실패한 경우
    if src is None:
        print('Image load failed!')
        return

    # FAST 검출기를 생성하고 임계값을 60으로 설정
    fast = cv2.FastFeatureDetector_create(60)
    # FAST 검출기를 사용하여 키포인트를 검출
    keypoints = fast.detect(src)

    # 결과를 표시하기 위해 그레이스케일 이미지를 BGR로 변환
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

    # 검출된 키포인트마다 빨간 원으로 표시
    for kp in keypoints:
        pt = (int(kp.pt[0]), int(kp.pt[1]))
        cv2.circle(dst, pt, 5, (0, 0, 255), 2)

    # 원본 이미지와 코너점이 표시된 이미지를 화면에 표시
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

# 메인 함수
if __name__ == "__main__":
    # Harris 코너 검출 함수 호출
    corner_harris()
    # FAST 코너 검출 함수 호출
    corner_fast()


In [12]:
import sys
import numpy as np
import cv2

# 입력 이미지 파일을 그레이스케일로 읽어옵니다.
src = cv2.imread('../data/img/malle_girl.png', cv2.IMREAD_GRAYSCALE)

# 이미지를 읽어오는 데 실패했을 경우 에러 메시지를 출력하고 프로그램을 종료합니다.
if src is None:
    print('Image load failed!')
    sys.exit()

# ORB 검출기 객체를 생성합니다.
orb = cv2.ORB_create()

# ORB를 사용하여 특징점을 검출합니다.
keypoints = orb.detect(src)

# 검출된 특징점들에 대해 기술자(Descriptor)를 계산합니다.
keypoints, desc = orb.compute(src, keypoints)

# 검출된 특징점의 개수를 출력합니다.
print('len(keypoints):', len(keypoints))

# 계산된 기술자의 크기(차원)를 출력합니다.
print('desc.shape:', desc.shape)

# 검출된 특징점을 원본 이미지 위에 그립니다. 
# cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS 플래그는 키포인트의 크기와 방향을 나타내는 원을 그리도록 합니다.
dst = cv2.drawKeypoints(src, keypoints, None, (-1, -1, -1),
                       cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

# 원본 이미지를 화면에 출력합니다.
cv2.imshow('src', src)

# 특징점이 그려진 이미지를 화면에 출력합니다.
cv2.imshow('dst', dst)

# 키 입력을 대기하고, 아무 키나 누르면 창을 닫습니다.
cv2.waitKey()
cv2.destroyAllWindows()


Image load failed!


SystemExit: 

c:\anaconda3\envs\cv2\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
import numpy as np
import cv2


def keypoint_matching():
    src1 = cv2.imread('../data/img/cropped.png', cv2.IMREAD_GRAYSCALE)
    src2 = cv2.imread('../data/img/bbb.png', cv2.IMREAD_GRAYSCALE)

    if src1 is None or src2 is None:
        print('Image load failed!')
        return

    orb = cv2.ORB_create()

    keypoints1, desc1 = orb.detectAndCompute(src1, None)
    keypoints2, desc2 = orb.detectAndCompute(src2, None)
    print('desc1.shape:', desc1.shape)
    print('desc2.shape:', desc2.shape)

    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING)
    matches = matcher.match(desc1, desc2)

    dst = cv2.drawMatches(src1, keypoints1, src2, keypoints2, matches, None)

    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


def good_matching():
    src1 = cv2.imread('../data/img/cropped.png', cv2.IMREAD_GRAYSCALE)
    src2 = cv2.imread('../data/img/bbb.png', cv2.IMREAD_GRAYSCALE)

    if src1 is None or src2 is None:
        print('Image load failed!')
        return

    orb = cv2.ORB_create()

    keypoints1, desc1 = orb.detectAndCompute(src1, None)
    keypoints2, desc2 = orb.detectAndCompute(src2, None)

    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING)
    matches = matcher.match(desc1, desc2)

    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = matches[:50]

    dst = cv2.drawMatches(src1, keypoints1, src2, keypoints2, good_matches, None,
                         flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


def find_homography():
    src1 = cv2.imread('../data/img/cropped.png', cv2.IMREAD_GRAYSCALE)
    src2 = cv2.imread('../data/img/bbb.png', cv2.IMREAD_GRAYSCALE)

    if src1 is None or src2 is None:
        print('Image load failed!')
        return

    orb = cv2.ORB_create()

    keypoints1, desc1 = orb.detectAndCompute(src1, None)
    keypoints2, desc2 = orb.detectAndCompute(src2, None)

    matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING)
    matches = matcher.match(desc1, desc2)

    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = matches[:50]

    dst = cv2.drawMatches(src1, keypoints1, src2, keypoints2, good_matches, None,
                         flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    pts1 = np.array([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2).astype(np.float32)
    pts2 = np.array([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2).astype(np.float32)

    H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC)

    (h, w) = src1.shape[:2]
    corners1 = np.array([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2).astype(np.float32)
    corners2 = cv2.perspectiveTransform(corners1, H)
    corners2 = corners2 + np.float32([w, 0])

    cv2.polylines(dst, [np.int32(corners2)], True, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    keypoint_matching()
    good_matching()
    find_homography()


desc1.shape: (123, 32)
desc2.shape: (500, 32)
